In [1]:
import numpy as np

## Метод Ньютона

In [2]:
def calc_deriv(f):
    return f[:-1] * np.arange(len(f) - 1, 0, -1)

def newton_method(x0, eps, f):
    k = 1
    
    xk = x0
    xk_next = xk - (np.poly1d(f)(xk) / np.poly1d(calc_deriv(f))(xk)) 
    
    while abs(xk_next - xk) > eps:
        xk = xk_next
        xk_next = xk - (np.poly1d(f)(xk) / np.poly1d(calc_deriv(f))(xk))
        k += 1
    
    return {'x': xk_next, 'depth': k}

def solve_with_newton(eps, f):
    for x in sorted(f, reverse=True):
        response = newton_method(x, eps, f)
        if np.isclose(np.poly1d(f)(response['x']), 0., atol=eps):
            yield response | {'x0' : x}

## Упрощенный метод Ньютона

In [3]:
def simplify_newton_method(x0, eps, f):
    k = 1
    
    xk = x0
    f0_deriv = np.poly1d(calc_deriv(f))
    xk_next = xk - (np.poly1d(f)(xk) / f0_deriv(x0))
    
    func = np.poly1d(f)
    
    while abs(func(xk_next)) > eps:
        xk = xk_next
        xk_next = xk - (func(xk) / f0_deriv(x0))
        k += 1
        
    return {'x': xk_next, 'depth': k}

def solve_with_simplified_newton(eps, f):
    for x in sorted(f, reverse=True):
        response = simplify_newton_method(x, eps, f)
        if np.isclose(np.poly1d(f)(response['x']), 0., atol=eps):
            yield response | {'x0' : x}


## Метод Ньютона-Бройдена

In [4]:
def newton_broyden_method(x0, eps, f):
    k = 1
    ck = 1
    
    xk = x0
    xk_next = xk - ck * (np.poly1d(f)(xk) / np.poly1d(calc_deriv(f))(xk))
    
    delta = abs(xk_next - xk)
    while delta > eps:
        if abs(np.poly1d(f)(xk)) >  abs(np.poly1d(f)(xk_next)):
            ck = 0.9
        else:
            ck = 1.2
        xk = xk_next
        xk_next = xk - ck * (np.poly1d(f)(xk) / np.poly1d(calc_deriv(f))(xk))
#         if abs(xk_next - xk) / delta > 0.5:
#             ck = 2
#         elif abs(np.poly1d(f)(xk)) > abs():
#             ck = 1
#         else:
#             ck = 0.7
        delta = abs(xk_next - xk)
        k += 1
        
    return {'x': xk_next, 'depth': k}

## Метод секущих

$$ x^{(k+1)} = x^{(k)} - \frac{f(x^{(k)})}{f(x^{(k)}) - f(x^{(k-1)})}\cdot(x^{(k)} - x^{(k-1)}) $$

In [5]:
def secant_method(x0, eps, delta, f):
    k = 1
    func = np.poly1d(f)
    
    xk_prev = x0
    xk = x0 - (func(x0) / ((func(x0) - func(x0 - delta)) / delta))
    xk_next = xk - (func(xk) / (func(xk) - func(xk_prev))) * (xk - xk_prev)
    
    while abs(xk_next - xk) > eps:
        xk_prev = xk
        xk = xk_next
        xk_next = xk - (func(xk) / (func(xk) - func(xk_prev))) * (xk - xk_prev)
        
        k += 1
    
    return {'x': xk_next, 'depth': k}

### Пример работы
$$ 3x^4 - 4x^3 - 8x^2 + 10x - 7 = 0 $$

In [6]:
eps = 1e-3
delta = 1e-1
coeff = np.array([3, -4, -8, 10, -7])
response = list(solve_with_newton(eps, coeff))[1]
print(f"Метод Ньютона: x = {round(response['x'], 4)}, x0 = {response['x0']}, количество итераций = {response['depth']}")
response = list(solve_with_simplified_newton(eps, coeff))[0]
print(f"Упрощённый метод Ньютона: x = {round(response['x'], 4)}, x0 = {response['x0']}, количество итераций = {response['depth']}")
x0 = 3
response = newton_broyden_method(x0, eps, coeff)
print(f"Метод Ньютона-Бройдена: x = {round(response['x'], 4)}, x0 = {x0}, количество итераций = {response['depth']}")
x0 = 10
response = newton_broyden_method(x0, eps, coeff)
print(f"Метод секущих: x = {round(response['x'], 4)}, x0 = {x0}, количество итераций = {response['depth']}")

Метод Ньютона: x = 2.0994, x0 = 3, количество итераций = 5
Упрощённый метод Ньютона: x = 2.0994, x0 = 10, количество итераций = 2997
Метод Ньютона-Бройдена: x = 2.0995, x0 = 3, количество итераций = 6
Метод секущих: x = 2.0994, x0 = 10, количество итераций = 12


### Теоремы

#### Об оценке модулей корней уравнения

In [7]:
def est_modulus(coeffs: np.array):
    A = np.max(coeffs)
    B = np.max(coeffs[:-1])
    
    return (1 / (1 + B / coeffs[-1]), 1 + A / coeffs[0])

#### теорема Лагранжа о верхней границе положительных корней уравнения

In [8]:
def lagrange_sup(coeffs: np.array):
    if coeffs[0] < 0:
        return None
    return 1 + np.power(np.abs(np.min(coeffs[coeffs < 0])) / coeffs[0], 1 / (len(coeffs) - 1))

#### о нижних и верхних границах положетельных и отрицательных корней алгебраического уравнения

In [9]:
def sup_and_inf(coeffs: np.array):
    def negative_transform(polynom: np.array):
        n = len(polynom)
        return np.array([polynom[n - i - 1] if i % 2 == 0 else -polynom[n - i - 1] for i in range(n - 1, -1, -1)])
    
    R = lagrange_sup(coeffs)
    R1 = lagrange_sup(coeffs[::-1])
    R2 = lagrange_sup(negative_transform(coeffs))
    R3 = lagrange_sup(negative_transform(coeffs)[::-1])
    if None not in (R, R1, R2, R3):
        return {"x+": (1 / R1, R), "x-": (-R2, -1 / R3)}
    else:
        return None

#### теорема Декарта о количестве действительных корней алгебраических уравнений

In [10]:
def descartes_count(coeff: np.array):
    def negative_transform(polynom: np.array):
        n = len(polynom)
        return np.array([polynom[n - i - 1] if i % 2 == 0 else -polynom[n - i - 1] for i in range(n - 1, -1, -1)])
    
    count_positive = 0
    for i in range(len(coeff) - 1):
        if coeff[i] * coeff[i+1] < 0:
            count_positive += 1
    
    count_negative = 0
    coeff_transformed = negative_transform(coeff)
    for i in range(len(coeff_transformed) - 1):
        if coeff_transformed[i] * coeff_transformed[i+1] < 0:
            count_negative += 1
    
    return {"S1": count_positive, "S2": count_negative}

#### теорема Гюа о необходимом условии действительности всех корней алгебраического уравнения

In [11]:
def is_real_roots(coeff: np.array):
    for i in range(1, len(coeff)-1):
        if coeff[i] ** 2 <= coeff[i-1] * coeff[i+1]:
            return False
    return True

#### теорема для отделения корней

In [12]:
def selection_interval(coeff: np.array, a=None, b=None):
    f = np.poly1d(coeff)
    
    if None not in [a, b]:
        if f(a) * f(b) < 0:
            return True
        else:
            return False
        
    for i in range(100):
        a, b = np.random.randint(np.min(coeff), np.max(coeff), 2)
        if f(a) * f(b) < 0:
            return a, b

### Пример работы
$$ 3x^4 - 4x^3 - 8x^2 + 10x + 7 = 0 $$

In [13]:
coeff = np.array([3, -4, -8, 10, 7])
response = est_modulus(coeff)
print(f'Теорема 3.3: {np.round(response[0], 3)} < |x| <= {np.round(response[1], 3)}')
print(f'Теорема 3.4: R = {np.round(lagrange_sup(coeff), 3)}')
response = sup_and_inf(coeff)
print(f'Теорема 3.5: {np.round(response["x+"][0], 3)} <= x+ <= {np.round(response["x+"][1], 3)}  {np.round(response["x-"][0], 3)} <= x- <= {np.round(response["x-"][1], 3)}')
response = descartes_count(coeff)
print(f'Теорема 3.6: S1 = {response["S1"]}, S2 = {response["S2"]}')
print(f'Теорема 3.7: Все корни действительные? - {is_real_roots(coeff)}')
response = selection_interval(coeff)
print(f'Теорема 3.8: Минимум один корень находиться на промежутке [{response[0]}, {response[1]}]')

Теорема 3.3: 0.412 < |x| <= 4.333
Теорема 3.4: R = 2.278
Теорема 3.5: 0.492 <= x+ <= 2.278  -2.351 <= x- <= -0.478
Теорема 3.6: S1 = 2, S2 = 2
Теорема 3.7: Все корни действительные? - True
Теорема 3.8: Минимум один корень находиться на промежутке [-3, -1]
